<img src='img/header.png'>

Hyper Parameter Optimization (HPO) imporves model quality by searching the space of possible 'architecture parameters,' parameters not usually trained during the learning process.  This search can significantly boost model quality relative to default parameters and non-expert tuning; however, the search over architectures can take a very long time on a non-accelerated platform. In this notebook, we containerize a RAPIDS workflow and run Bring-Yor-Own-Container SageMaker HPO to show how we can overcome the computational complexity of model search. 

We accelerate HPO in two key ways: 
* by *scaling within a node* (e.g., multi-GPU where each GPU brings a magnitude higher core count relative to CPUs), and 
* by *scaling across nodes* and running parallel trials on cloud instances.

By combining these two powers HPO experiments that feel unapproachable and may take multiple days on CPU instances can complete in just hours. For example, we find a <span style="color:#8735fb; font-size:14pt"> **12X** </span> speedup in wall clock time and a <span style="color:#8735fb; font-size:14pt"> **4.5x** </span> reduction in cost when comparing between GPU and CPU [EC2 Spot instances](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html) on 100 XGBoost HPO trials using 10 parallel workers on 10 years of the Airline Dataset (~63M flights). For additional details refer to the <a href='#experiments'>end of the notebook</a>.

With all these powerful tools at our disposal, every data scientist should feel empowered to uplevel their model before serving it to the world!

In [1]:
import sagemaker
from helper_functions import *
working_directory = get_notebook_path()

<img src='img/hpo.png'>

<span style="color:#8735fb; font-size:22pt"> **Key Choices** </span>

Let's go ahead and choose the configuration options for our HPO run.

Note that we've set the configuration defaults to values that are on the low end, however you are welcome to scale them up. 

<span style="color:#8735fb; font-size:18pt"> [ Dataset and S3 Bucket ] </span>

We target a large real-world structured dataset of flight logs for US airlines ( published monthly since 1987 by the Bureau of Transportation [dataset link](https://www.transtats.bts.gov/DatabaseInfo.asp?DB_ID=120&DB_URL=), additional details in <a href='#dataset'>Section 1.1</a>). 

We offer several sizes of this dataset in <a href='https://parquet.apache.org/'>parquet</a> (compressed column storage) format: `1_year` (2019, 7.2M flights), `3_year` (2016-2019, 18M flights) or `10_year` (2009-2019, 63M flights). 

In [2]:
dataset_directory = '1_year'

assert( dataset_directory in [ '1_year', '3_year', '10_year'] )

For demo purposes we host these pre-built datasets in a public S3 bucket, you are of course welcome to copy the data into the corresponding region of your compute, or host your own dataset in a bucket of your choosing.

In [3]:
bucket = 'sagemaker-rapids-hpo-us-east-1'

s3_data_input = f"s3://{bucket}/{dataset_directory}"
s3_model_output = f"s3://{bucket}/trained-models"

<span style="color:#8735fb; font-size:18pt"> [ Algorithm ] </span>

From a ML/algorithm perspective, we offer [XGBoost](https://xgboost.readthedocs.io/en/latest/#) and [RandomForest](https://docs.rapids.ai/api/cuml/stable/cuml_blogs.html#tree-and-forest-models) decision tree models which do quite well on this structured dataset.

In [4]:
algorithm_choice = 'XGBoost'

assert ( algorithm_choice in [ 'XGBoost', 'RandomForest' ])

We can also optionally increase robustness via reshuffles of the train-test split (i.e., [cross-validation folds](https://scikit-learn.org/stable/modules/cross_validation.html)). Typical values here are between 3 and 10 folds.

In [5]:
cv_folds = 3

assert ( cv_folds >= 1 )

<span style="color:#8735fb; font-size:18pt"> [ Code ] </span>

We enable the option of running different code variations that unlock increasing amounts of parallelism in the compute workflow.

* `singleCPU`* = [pandas](https://pandas.pydata.org/) + [sklearn](https://scikit-learn.org/stable/)
* `multiCPU`   = [dask](https://dask.org/) + [pandas](https://pandas.pydata.org/) + [sklearn](https://scikit-learn.org/stable/)

* `singleGPU` = [cudf](https://github.com/rapidsai/cudf) + [cuml](https://github.com/rapidsai/cuml)
* `multiGPU`  = [dask](https://dask.org/) + [cudf](https://github.com/rapidsai/cudf) + [cuml](https://github.com/rapidsai/cuml) 

For some context, cuDF and cuML are part of the RAPIDS library ecosystem. cuDF is a GPU accelerated dataframe library aimed to mirror pandas, while cuML is a GPU accelerated machine-learning library aimed to mirror sklearn.

Note that the single-CPU option still uses multiple cores in the model training portion of the workflow; however, to unlock parallelism in each stage of the workflow we use [Dask](https://dask.org/). We've integrated all to of these options in the `rapids_cloud_ml.py` file. 



In [6]:
code_choice = 'singleGPU' 

assert ( code_choice in [ 'singleCPU', 'singleGPU', 'multiCPU', 'multiGPU'])

<span style="color:#8735fb; font-size:18pt"> [ Search Ranges and Strategy ] </span>
<a id='strategy-and-param-ranges'></a>

One of the most important choices when running HPO is to choose the bounds of the hyper-parameter search process. Below we've set the ranges of the hyper-parameters to allow for interesting variation, you are of course welcome to revise these ranges based on domain knowledge especially if you plan to plug in your own dataset. 

> Note that we support additional algorithm specific parameters (refer to the `parse_hyper_parameter_inputs` function in `rapids_cloud_ml.py`), but for demo purposes have limited our choice to the three parameters that overlap between the XGBoost and RandomForest algorithms. For more details see the documentation for [XGBoost parameters](https://xgboost.readthedocs.io/en/latest/parameter.html) and [RandomForest parameters](https://docs.rapids.ai/api/cuml/stable/api.html#random-forest).


In [7]:
hyperparameter_ranges = {
    'max_depth'    : sagemaker.parameter.IntegerParameter        ( 5, 15 ),
    'n_estimators' : sagemaker.parameter.IntegerParameter        ( 100, 500 ),
    'max_features' : sagemaker.parameter.ContinuousParameter     ( 0.1, 1.0 ),
    # please see note above for adding additional model specific parameters
}

In [8]:
if 'XGBoost' in algorithm_choice: 
    # number of trees parameter name difference b/w XGBoost and RandomForest
    hyperparameter_ranges['num_boost_round'] = hyperparameter_ranges.pop('n_estimators')

We can also choose between a Random and Bayesian search strategy for picking parameter combinations. 

**Random Search**: Choose a random combination of values from within the ranges for each training job it launches. The choice of hyperparameters doesn't depend on previous results so you can run the maximum number of concurrent workers without affecting the performance of the search. 

**Bayesian Search**: Make a guess about which hyperparameter combinations are likely to get the best results. After testing the first set of hyperparameter values, hyperparameter tuning uses regression to choose the next set of hyperparameter values to test.

In [9]:
search_strategy = 'Random'

assert ( search_strategy in [ 'Random', 'Bayesian' ])

<span style="color:#8735fb; font-size:18pt"> [ Experiment Scale ] </span>

We also need to decide how may total experiments to run, and how many should run in parallel. Below we have a very conservative number of maximum jobs to run so that you don't accidently spawn large computations when starting out, however for meaningful HPO searches this number should be much higher (e.g., in our experiments we often run 100 max_jobs). Note that you may need to request a [quota limit increase](https://docs.aws.amazon.com/general/latest/gr/sagemaker.html) for additional  `max_parallel_jobs` parallel workers. 

In [10]:
max_jobs = 8

In [11]:
max_parallel_jobs = 2

Let's also set the max duration for an individual job to 24 hours so we don't have run-away compute jobs taking too long.

In [12]:
max_duration_of_experiment_seconds = 60 * 60 * 24

<span style="color:#8735fb; font-size:18pt"> [ Compute Platform ] </span>

Based on the dataset size and compute choice we will try to recommend an instance choice*, you are of course welcome to select alternate configurations. 
> e.g., For the 10_year dataset option, we suggest ml.p3.8xlarge GPU instances and ml.m5.24xlarge CPU instances ( we'll need upwards of 200GB CPU RAM during model training).

In [13]:
instance_type = recommend_instance_type ( code_choice, dataset_directory  ) 

recommended instance type : ml.p3.2xlarge 
instance details          : 1x V100, 16GB GPU memory, 61GB CPU memory


In addition to choosing our instance type, we can also enable significant savings by leveraging [AWS EC2 Spot Instances](https://aws.amazon.com/ec2/spot/). We highly recommend that you enable this choice as it typically leads to 60-70% cost savings.

In [14]:
use_spot_instances_flag = True

<span style="color:#8735fb; font-size:22pt"> **Validate** </span>

In [15]:
summarize_choices( s3_data_input, s3_model_output, code_choice, algorithm_choice, 
                   cv_folds, instance_type, use_spot_instances_flag, search_strategy, 
                   max_jobs, max_parallel_jobs, max_duration_of_experiment_seconds )

s3 data input    =	s3://sagemaker-rapids-hpo-us-east-1/1_year
s3 model output  =	s3://sagemaker-rapids-hpo-us-east-1/trained-models
compute          =	singleGPU
algorithm        =	XGBoost, 3 cv-fold
instance         =	ml.p3.2xlarge
spot instances   =	True
hpo strategy     =	Random
max_experiments  =	8
max_parallel     =	2
max runtime      =	86400 sec


<span style="display: block; text-align: center; color:#8735fb; font-size:30pt"> **1. ML Workflow** </span>

<img src='img/ml_workflow.png' width='800'> 

<span style="color:#8735fb; font-size:20pt"> 1.1 - Dataset </span>
<a id ='dataset'></a>

In this demo we'll utilize the Airline dataset (Carrier On-Time Performance 1987-2020, available from the [Bureau of Transportation Statistics](https://transtats.bts.gov/Tables.asp?DB_ID=120&DB_Name=Airline%20On-Time%20Performance%20Data&DB_Short_Name=On-Time#)). 

The public dataset contains logs/features about flights in the United States (17 airlines) including:

* Locations and distance  ( `Origin`, `Dest`, `Distance` )
* Airline / carrier ( `Reporting_Airline` )
* Scheduled departure and arrival times ( `CRSDepTime` and `CRSArrTime` )
* Actual departure and arrival times ( `DpTime` and `ArrTime` )
* Difference between scheduled & actual times ( `ArrDelay` and `DepDelay` )
* Binary encoded version of late, aka our target variable ( `ArrDelay15` )

Using these features we'll build a classifier model to predict whether a flight is going to be more than 15 minutes late on arrival as it prepares to depart.

<span style="color:#8735fb; font-size:20pt"> 1.2 - Python ML Workflow </span>

To build a RAPIDS enabled SageMaker HPO we first need to build an Estimator. An Estimator is a container image that captures all the software needed to run an HPO experiment. The container is augmented with entrypoint code that will be trggered at runtime by each worker. The entrypoint code enables us to write custom models and hook them up to data. 

In order to work with SageMaker HPO, the entrypoint logic should parse hyper-parameters (supplied by AWS SageMaker), load and split data, build and train a model, score/evaluate the trained model, and emit an output representing the final score for the given hyper-parameter setting. We've already built multiple variations of this code.

If you would like to make changes by adding your custom model logic feel free to modify the **train.py** and **rapids_cloud_ml.py** files in the code directory. 
You are also welcome to uncomment the cells below to load the read/review the code.

In [16]:
# %load ../code/train.py

In [17]:
# %load ../code/rapids_cloud_ml.py

<span style="display: block; text-align: center; color:#8735fb; font-size:30pt"> **2. Build Estimator** </span>

<img src='img/estimator.png' width='800'>

As we've already mentioned, the SageMaker Estimator represents the containerized software stack that AWS SageMaker will replicate to each worker node.

The first step to building our Estimator, is to augment a RAPIDS container with our ML Workflow code from above, and push this image to Amazon Elastic Cloud Registry so it is available to SageMaker.

For additional options and details see the [Estimator documentation](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator).


<span style="color:#8735fb; font-size:20pt"> 2.1 - Containerize and Push to ECR </span>

Now lets turn to building our container so that it can integrate with the AWS SageMaker HPO API.

Our container can either be built on top of the latest RAPIDS [ nightly ] image as a starting layer or the RAPIDS stable image.


In [18]:
rapids_stable = 'rapidsai/rapidsai:0.14-cuda10.1-runtime-ubuntu18.04-py3.7'
rapids_nightly = 'rapidsai/rapidsai-nightly:0.15-cuda10.1-runtime-ubuntu18.04-py3.7'

rapids_base_container = rapids_stable
assert ( rapids_base_container in [ rapids_stable, rapids_nightly ] )

To get things rolling lets make sure we can query our AWS SageMaker execution role and session as well as our account ID and AWS region.

In [19]:
execution_role = sagemaker.get_execution_role()
session = sagemaker.Session()

account=!(aws sts get-caller-identity --query Account --output text)
region=!(aws configure get region)

In [20]:
account, region

(['561241433344'], ['us-east-1'])

Let's also decide on the full name of our container.

In [21]:
image_base = 'cloud-ml-sagemaker'
image_tag  = rapids_base_container.split(':')[1]

In [22]:
ecr_fullname = f"{account[0]}.dkr.ecr.{region[0]}.amazonaws.com/{image_base}:{image_tag}"

In [23]:
ecr_fullname

'561241433344.dkr.ecr.us-east-1.amazonaws.com/cloud-ml-sagemaker:0.14-cuda10.1-runtime-ubuntu18.04-py3.7'

<span style="color:#8735fb; font-size:18pt"> 2.1.1 - Write Dockerfile </span>

We write out the Dockerfile in this cell, write it to disk, and in the next cell execute the docker build command. 

First, lets switch our working directory to the location of the Estimator entypoint and library code.

In [24]:
%cd code

/home/ec2-user/SageMaker/rapids_bring_your_own/code


Let's now write our selected RAPDIS image layer as the first FROM statement in the the Dockerfile.

In [25]:
with open('Dockerfile', 'w') as dockerfile_handle: 
    dockerfile_handle.writelines( 'FROM ' + rapids_base_container + '\n')

Next lets write the remaining pieces of the Dockerfile, namely adding the sagemaker-training-toolkit and copying our python code.

In [26]:
%%writefile -a Dockerfile

# install https://github.com/aws/sagemaker-training-toolkit
RUN apt-get update && apt-get install -y --no-install-recommends build-essential \ 
    && source activate rapids && pip3 install sagemaker-training

# path where sagemaker looks for our code
ENV CLOUD_PATH="/opt/ml/code"

# copy our latest [local] code into the container 
COPY rapids_cloud_ml.py $CLOUD_PATH/rapids_cloud_ml.py
COPY train.py $CLOUD_PATH/train.py

# sagemaker entrypoint will be train.py
ENV SAGEMAKER_PROGRAM train.py 

WORKDIR $CLOUD_PATH

Appending to Dockerfile


Lastly, let's ensure that our Dockerfile correctly captured our base image selection.

In [27]:
validate_dockerfile( rapids_base_container )
!cat Dockerfile

FROM rapidsai/rapidsai:0.14-cuda10.1-runtime-ubuntu18.04-py3.7

# install https://github.com/aws/sagemaker-training-toolkit
RUN apt-get update && apt-get install -y --no-install-recommends build-essential \ 
    && source activate rapids && pip3 install sagemaker-training

# path where sagemaker looks for our code
ENV CLOUD_PATH="/opt/ml/code"

# copy our latest [local] code into the container 
COPY rapids_cloud_ml.py $CLOUD_PATH/rapids_cloud_ml.py
COPY train.py $CLOUD_PATH/train.py

# sagemaker entrypoint will be train.py
ENV SAGEMAKER_PROGRAM train.py 

WORKDIR $CLOUD_PATH


<span style="color:#8735fb; font-size:18pt"> 2.1.2 Build and Tag </span>

The build step will be dominated by the download of the RAPIDS image (base layer). If it's already been downloaded the build will take less than 1 minute.

In [28]:
!docker pull $rapids_base_container

0.14-cuda10.1-runtime-ubuntu18.04-py3.7: Pulling from rapidsai/rapidsai
Digest: sha256:200cbe0922b26cbddba2cfcf50a97ec91817c40767516fde45ebaaf801b7477d
Status: Image is up to date for rapidsai/rapidsai:0.14-cuda10.1-runtime-ubuntu18.04-py3.7
docker.io/rapidsai/rapidsai:0.14-cuda10.1-runtime-ubuntu18.04-py3.7


In [29]:
%%time
!docker build . -t $ecr_fullname -f Dockerfile

Sending build context to Docker daemon     64kB
Step 1/7 : FROM rapidsai/rapidsai:0.14-cuda10.1-runtime-ubuntu18.04-py3.7
 ---> 65edd55ef398
Step 2/7 : RUN apt-get update && apt-get install -y --no-install-recommends build-essential     && source activate rapids && pip3 install sagemaker-training
 ---> Using cache
 ---> 66c7e23b7262
Step 3/7 : ENV CLOUD_PATH="/opt/ml/code"
 ---> Using cache
 ---> f4171e572ab0
Step 4/7 : COPY rapids_cloud_ml.py $CLOUD_PATH/rapids_cloud_ml.py
 ---> 836af9aebea8
Step 5/7 : COPY train.py $CLOUD_PATH/train.py
 ---> fc5b3a3455f9
Step 6/7 : ENV SAGEMAKER_PROGRAM train.py
 ---> Running in f81e2ac42334
Removing intermediate container f81e2ac42334
 ---> 7493ced5e8ff
Step 7/7 : WORKDIR $CLOUD_PATH
 ---> Running in efb9ae3cf9b2
Removing intermediate container efb9ae3cf9b2
 ---> 2629cf8af0f9
Successfully built 2629cf8af0f9
Successfully tagged 561241433344.dkr.ecr.us-east-1.amazonaws.com/cloud-ml-sagemaker:0.14-cuda10.1-runtime-ubuntu18.04-py3.7
CPU times: user 22.7

<span style="color:#8735fb; font-size:18pt"> 2.1.3 - Publish to Elastic Cloud Registry (ECR) </span>

Now that we've built and tagged our container its time to push it to Amazon's container registry (ECR). Once in ECR, AWS SageMaker will be able to leverage our image to build Estimators and run experiments.


Docker Login to ECR

In [30]:
docker_login_str = !(aws ecr get-login --region {region[0]} --no-include-email)

In [31]:
!{docker_login_str[0]}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


Create ECR repository [ if it doesn't already exist]

In [32]:
repository_query = !(aws ecr describe-repositories --repository-names $image_base)
if repository_query[0] == '':
    !(aws ecr create-repository --repository-name $image_base)

Let's now actually push the container to ECR
> Note the first push to ECR may take some time (hopefully less than 10 minutes).

In [33]:
ecr_fullname

'561241433344.dkr.ecr.us-east-1.amazonaws.com/cloud-ml-sagemaker:0.14-cuda10.1-runtime-ubuntu18.04-py3.7'

In [34]:
!docker push $ecr_fullname

The push refers to repository [561241433344.dkr.ecr.us-east-1.amazonaws.com/cloud-ml-sagemaker]

9cc8bae6: Preparing 
407e4868: Preparing 
ce92dc39: Preparing 
7905ac2b: Preparing 
f23a2c1c: Preparing 
3564a770: Preparing 
fea6d95e: Preparing 
74f76be4: Preparing 
d332a58a: Preparing 
f11cbf29: Preparing 
a4b22186: Preparing 
afb09dc3: Preparing 
b5a53aac: Preparing 
c8e5063e: Preparing 
07e4868: Pushed lready exists 7kB4A0.14-cuda10.1-runtime-ubuntu18.04-py3.7: digest: sha256:425bae3a186dc3befe9cd577b095ea7bc382bfb81f9cf8b96e356f4bc92e8c89 size: 3476


<span style="color:#8735fb; font-size:20pt"> 2.2 - Create Estimator </span>

Having built our container [ +custom logic] and pushed it to ECR, we can finally compile all of efforts into an Estimator instance.

In [35]:
estimator_params = {
    'image_name' : ecr_fullname,
    
    'train_use_spot_instances': use_spot_instances_flag,
    'train_instance_type' : instance_type, 
    'train_instance_count' : 1,     
    
    'train_max_run'  : max_duration_of_experiment_seconds, # 24 hours 
    'train_max_wait' : max_duration_of_experiment_seconds + 1,  
    
    'input_mode'  : 'File',
    'output_path' : s3_model_output,
    
    'sagemaker_session' : session,     
    'role' : execution_role,    
}

In [36]:
estimator = sagemaker.estimator.Estimator( **estimator_params  )

<span style="color:#8735fb; font-size:20pt"> 2.3 - Test Estimator </span>

Now we are ready to test by asking SageMaker to run the BYOContainer logic inside our Estimator. This is a useful step if you've made changes to your custom logic and are interested in making sure everything works before launching a large HPO search. 

> Note: This verification step will use the default hyper-parameter values declared in our custom train code, as SageMaker HPO will not be orchestrating a search for this single run.

In [37]:
summarize_choices( s3_data_input, s3_model_output, code_choice, algorithm_choice, 
                   cv_folds, instance_type, use_spot_instances_flag, search_strategy, 
                   max_jobs, max_parallel_jobs, max_duration_of_experiment_seconds )

s3 data input    =	s3://sagemaker-rapids-hpo-us-east-1/1_year
s3 model output  =	s3://sagemaker-rapids-hpo-us-east-1/trained-models
compute          =	singleGPU
algorithm        =	XGBoost, 3 cv-fold
instance         =	ml.p3.2xlarge
spot instances   =	True
hpo strategy     =	Random
max_experiments  =	8
max_parallel     =	2
max runtime      =	86400 sec


In [38]:
assert ( input('confirm test run? [ y / n ] : ').lower() == 'y' )

job_name = new_job_name_from_config( dataset_directory, code_choice, 
                                     algorithm_choice, cv_folds,
                                     instance_type  )

estimator.fit(inputs = s3_data_input, job_name=job_name.lower())

confirm test run? [ y / n ] :  y


generated job name : 1y-sGPU-XGB-3cv-p3-2xlarge-dfc51

2020-08-03 20:32:23 Starting - Starting the training job...
2020-08-03 20:32:25 Starting - Launching requested ML instances......
2020-08-03 20:33:41 Starting - Preparing the instances for training......
2020-08-03 20:34:54 Downloading - Downloading input data
2020-08-03 20:34:54 Training - Downloading the training image..................
2020-08-03 20:37:52 Training - Training image download completed. Training in progress..2020-08-03 20:37:52,771 sagemaker-training-toolkit INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "framework_module": null,
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {},
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {
        "training": {
            "TrainingInputMode": "File",
            "S3Distributi

<span style="display: block; text-align: center; color:#8735fb; font-size:30pt"> **3. Run HPO** </span>

With a working SageMaker Estimator in hand, the hardest part is behind us. In the key choices section we <a href='#strategy-and-param-ranges'>already defined our search strategy and hyper-parameter ranges</a>, so all that remains is to choose a metric to evaluate performance on. For more documentation check out the AWS SageMaker [HyperParameter Tuner documentation](https://sagemaker.readthedocs.io/en/stable/tuner.html).

<img src='img/run_hpo.png'>

<span style="color:#8735fb; font-size:20pt"> 3.1 - Define Metric </span>

We only focus on a single metric, which we call 'final-score', that captures the accuracy of our model on the test data unseen during training. You are of course welcome to add aditional metrics, see [AWS SageMaker documentation on Metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-metrics.html). When defining a metric we provide a regular expression (i.e., string parsing rule) to extract the key metric from the output of each Estimator/worker.

In [39]:
metric_definitions = [{'Name': 'final-score', 'Regex': 'final-score: (.*);'}]

In [40]:
objective_metric_name = 'final-score'

<span style="color:#8735fb; font-size:20pt"> 3.2 - Define Tuner </span>

Finally we put all of the elements we've been building up together into a HyperparameterTuner declaration. 

In [41]:
hpo = sagemaker.tuner.HyperparameterTuner( estimator = estimator,
                                           metric_definitions = metric_definitions, 
                                           objective_metric_name = objective_metric_name,
                                           objective_type = 'Maximize',
                                           hyperparameter_ranges = hyperparameter_ranges,
                                           strategy = search_strategy,  
                                           max_jobs = max_jobs,
                                           max_parallel_jobs = max_parallel_jobs)

<span style="color:#8735fb; font-size:20pt"> 3.3 - Run HPO </span>

In [42]:
summarize_choices( s3_data_input, s3_model_output, code_choice, algorithm_choice, 
                   cv_folds, instance_type, use_spot_instances_flag, search_strategy, 
                   max_jobs, max_parallel_jobs, max_duration_of_experiment_seconds )

s3 data input    =	s3://sagemaker-rapids-hpo-us-east-1/1_year
s3 model output  =	s3://sagemaker-rapids-hpo-us-east-1/trained-models
compute          =	singleGPU
algorithm        =	XGBoost, 3 cv-fold
instance         =	ml.p3.2xlarge
spot instances   =	True
hpo strategy     =	Random
max_experiments  =	8
max_parallel     =	2
max runtime      =	86400 sec


Let's be sure we take a moment to confirm before launching all of our HPO experiments. Depending on your configuration options running this cell can kick off a massive amount of computation!
> Once this process begins, we recommend that you use the SageMaker UI to keep track of the <a href='../img/gpu_hpo_100x10.png'>health of the HPO process and the individual workers</a>.

In [43]:
assert ( input('confirm HPO launch? [ y / n ] : ').lower() == 'y' )

tuning_job_name = new_job_name_from_config( dataset_directory, code_choice, 
                                            algorithm_choice, cv_folds, 
                                            instance_type )
hpo.fit( inputs = s3_data_input, 
         job_name = tuning_job_name, 
         wait = True, logs = 'All')

hpo.wait() # block until the .fit call above is completed

confirm HPO launch? [ y / n ] :  y


generated job name : 1y-sGPU-XGB-3cv-p3-2xlarge-b4b5d

.............................................................................................................................................................................................................................................................................................................................................................................!


<span style="color:#8735fb; font-size:20pt"> 3.4 - Results and Summary </span>

Once your job is complete there are multiple ways to analyze the results.
Below we display the performance of the best job, as well printing each HPO trial/job as a row of a dataframe.

In [44]:
hpo_results = summarize_hpo_results ( tuning_job_name )

best score: 0.9360807538032532
best params: {'max_depth': '6', 'max_features': '0.704100184794978', 'num_boost_round': '293'}
best job-name: 1y-sGPU-XGB-3cv-p3-2xlarge-b4b5d-007-46dea283


In [45]:
sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name).dataframe()

,max_depth,max_features,num_boost_round,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,13.0,0.364925,305.0,1y-sGPU-XGB-3cv-p3-2xlarge-b4b5d-008-a4d0d5b6,Completed,0.930250,2020-08-03 21:09:43+00:00,2020-08-03 21:14:01+00:00,258.0
1,6.0,0.704100,293.0,1y-sGPU-XGB-3cv-p3-2xlarge-b4b5d-007-46dea283,Completed,0.936081,2020-08-03 21:08:35+00:00,2020-08-03 21:12:18+00:00,223.0
2,11.0,0.386082,385.0,1y-sGPU-XGB-3cv-p3-2xlarge-b4b5d-006-d8fc5553,Completed,0.931477,2020-08-03 21:02:28+00:00,2020-08-03 21:06:59+00:00,271.0
3,9.0,0.877136,252.0,1y-sGPU-XGB-3cv-p3-2xlarge-b4b5d-005-e7e3c8f4,Completed,0.934227,2020-08-03 21:01:13+00:00,2020-08-03 21:05:34+00:00,261.0
4,13.0,0.310162,484.0,1y-sGPU-XGB-3cv-p3-2xlarge-b4b5d-004-8a3d1501,Completed,0.930274,2020-08-03 20:54:41+00:00,2020-08-03 20:59:35+00:00,294.0
5,11.0,0.303105,471.0,1y-sGPU-XGB-3cv-p3-2xlarge-b4b5d-003-12cb8f08,Completed,0.931133,2020-08-03 20:53:31+00:00,2020-08-03 20:57:58+00:00,267.0
6,11.0,0.965175,253.0,1y-sGPU-XGB-3cv-p3-2xlarge-b4b5d-002-2dea4d7c,Completed,0.932481,2020-08-03 20:46:45+00:00,2020-08-03 20:50:51+00:00,246.0
7,15.0,0.370175,180.0,1y-sGPU-XGB-3cv-p3-2xlarge-b4b5d-001-50986a4f,Completed,0.930297,2020-08-03 20:46:44+00:00,2020-08-03 20:51:52+00:00,308.0


 For a more in depth look at the HPO process we invite you to check out the <a href='https://github.com/awslabs/amazon-sagemaker-examples/tree/master/hyperparameter_tuning/analyze_results'>HPO_Analyze_TuningJob_Results.ipynb</a> notebook which shows how we can explore interesting things like the <a href='img/results_analysis.png'>impact of each individual hyper-parameter on the performance metric</a>.

<span style="color:#8735fb; font-size:20pt"> 3.5 - Getting the best Model </span>

Next let's download the best trained model from our HPO runs.

In [46]:
working_directory

'/home/ec2-user/SageMaker/rapids_bring_your_own'

In [47]:
local_filename = download_best_model( bucket, s3_model_output, hpo_results, working_directory )

Successfully downloaded best model
> filename: best_model.tar.gz
> local directory : /home/ec2-user/SageMaker/rapids_bring_your_own

full S3 path : s3://sagemaker-rapids-hpo-us-east-1/trained-models/1y-sGPU-XGB-3cv-p3-2xlarge-b4b5d-007-46dea283/output/trained-models/1y-sGPU-XGB-3cv-p3-2xlarge-b4b5d-007-46dea283/output/model.tar.gz


<span style="color:#8735fb; font-size:20pt"> 3.6 - Model Serving </span>

With your best model in hand, you can now move on to serving this model. 

The [SageMaker Inference Toolkit](https://github.com/aws/sagemaker-inference-toolkit) implements a model serving stack and can be easily added to any Docker container, making it deployable to SageMaker. This library's serving stack is built on Multi Model Server, and it can serve your own models or those you trained on SageMaker using machine learning frameworks with native SageMaker support. 

We'll leave you with pointers to documentation in case you want to go further, however a full implementation is out of scope for our HPO notebook. 

> Note that the best model we just downloaded is stored in a [Treelite](https://treelite.readthedocs.io/en/latest/) format and can run optimized inference using the GPU or the CPU.

<span style="color:#8735fb; font-size:20pt"> Summary </span>

We've now succesfully built a RAPIDS ML workflow, containerized it (as a SageMaker Estimator), and launched a set of HPO experiments to find the best hyper-paramters for our model.

If you are curious to go further, we invite you to plug in your own dataset and tweak the configuration settings to find your champion model!

**HPO Experiment Details**
<a id='experiments'></a>

As mentioned in the introduction we find a <span style="color:#8735fb; font-size:14pt"> **12X** </span> speedup in wall clock time and a <span style="color:#8735fb; font-size:14pt"> **4.5x** </span> reduction in cost when comparing between GPU and CPU instances on 100 HPO trials using 10 parallel workers on 10 years of the Airline Dataset (~63M flights). In these experiments we used the XGBoost algorithm with the multi-GPU vs multi-CPU Dask cluster and 10 cross validaiton folds. Below we offer a table with additional details.

<img src='img/results.png' width='70%'>

In the case of the CPU runs, 12 jobs were stopped since they exceeded the 24 hour limit we set. <a href='img/cpu_hpo_100x10.png'>CPU Job Summary Image</a>

In the case of the GPU runs, no jobs were stopped. <a href='img/gpu_hpo_100x10.png'>GPU Job Summary Image</a>

Note that in both cases 1 job failed because a spot instance was terminated. But 1 failed job out of 100 is a minimal tradeoff for the significant cost savings.

<span style="color:#8735fb; font-size:20pt"> Rapids References </span>


> [cloud-ml-examples](http://github.com/rapidsai/cloud-ml-examples)

> [RAPIDS HPO](https://rapids.ai/hpo)

> [cuML Documentation](https://docs.rapids.ai/api/cuml/stable/)

<span style="color:#8735fb; font-size:20pt"> SageMaker References </span>

> [SageMaker Training Toolkit](https://github.com/aws/sagemaker-training-toolkit)

> [Estimator Parameters](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

> Spot Instances [docs](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html), and [blog]()